<a href="https://colab.research.google.com/github/hhnafis/Function_calling-Tool_calling_in_LLM/blob/main/Function_calling_Tool_calling_in_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gemini API: Function calling config

In [1]:
!pip install -U -q "google-generativeai>=0.7.2"

In [3]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY_2"))

#Set up a model with tools

In [5]:
def enable_lights():
  """Turn on the lighting system."""
  print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex:str):
  """Set the light color. Lights must be enabled for this to work."""
  print(f"LIGHTBOT: Lights set to {rgb_hex}.")

def stop_lights():

  """Stop flashing lights."""
  print("LIGHTBOT: Lights turned offf.")

light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro",
    tools=light_controls,
    system_instruction=instruction
)
chat = model.start_chat()

##Create a helper function for setting `function_calling_config` on `tool_config`.

In [10]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

#Text-only mode: `NONE`

In [18]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello light-bot, what can you do?", tool_config=tool_config
)
print(response.text)

I can turn lights on and off, and I can set the color of the lights.



#Automatic mode: `AUTO`

In [16]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts[0])
chat.rewind();  # You are not actually calling the function, so remove this from the history.

function_call {
  name: "enable_lights"
  args {
  }
}



#Function-calling mode: `ANY`

In [19]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place PURPLE!", tool_config=tool_config)
print(response.parts[0])

function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "800080"
      }
    }
  }
}



#Automatic function calling
##`tool_config `works when enabling automatic function calling too.

In [20]:
available_fns = ["enable_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
auto_chat.send_message("It's awful dark in here...", tool_config=tool_config)

LIGHTBOT: Lights enabled.


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Let there be light!\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.05619437495867411
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 169,
        "candidates_token_count": 6,
        "total_token_count": 175
      }
    }),
)

#Example API for lighting control

In [37]:
import typing
def set_light_values(brightness: int, color_temp: typing.Literal["daylight", "cool", "warm"] = "warm"):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`. Defaults to "warm".

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

#Declare functions during model initialization

In [38]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])

#Generate a function call


In [40]:
chat = model.start_chat()
response = chat.send_message('Dim the lights so the room feels cozy and warm.')
response.parts


[function_call {
  name: "set_light_values"
  args {
    fields {
      key: "color_temp"
      value {
        string_value: "warm"
      }
    }
    fields {
      key: "brightness"
      value {
        number_value: 30
      }
    }
  }
}
]

#Example for Parallel function calling

In [30]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [32]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(energetic=True, loud=True, bpm=120.0)
dim_lights(brightness=0.3)

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=120.0
Lights are now set to 30%


True

In [33]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

OK. I've turned on the disco ball, started playing some energetic music at 120 bpm, and dimmed the lights to 50% brightness.  Let's get this party started!

